#### IMPORTAMOS LIBRERIAS

In [73]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import random
from time import sleep
import pandas as pd

#### FUNCION PARA OBTENER CREDENCIALES

In [74]:
def get_credentials():
    credentials = open('credentials.txt').readline().strip().split(',')
    user = credentials[0]
    password = credentials[1]
    
    return user, password

#### CREAMOS EL DRIVER

In [75]:

def get_driver():
    options = Options()
    #options.add_argument(
    #    "user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/71.0.3578.80 Chrome/71.0.3578.80 Safari/537.36"
    #)
    options.add_argument("--disable-notifications")
    options.add_argument("--headles")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-software-rasterizer")
    options.add_argument('--log-level=3')
    options.add_argument("--lang=es-ES")

    options.add_experimental_option('excludeSwitches', ['enable-logging'])


    driver = webdriver.Chrome('./chromedriver.exe', options = options)
    driver.maximize_window()
    driver.get('https://www.instagram.com/')
    
    
    sleep(random.uniform(3,4)) 

    return driver

#### FUNCIONES PRINCIPALES PARA SCRAPEAR

In [76]:
def cleaning_text(text):
    clean_text = text.replace('\n', ' ').replace('\r', ' ').replace('\t',' ').replace(',','').replace('~','').strip()
    return clean_text


def logging_instagram(driver):
    username, password = get_credentials()
    username_input = WebDriverWait(driver, 10).until(   
            EC.presence_of_element_located((By.XPATH, '//input[@name="username"]'))
        )
    password_input =  WebDriverWait(driver, 10).until(   
            EC.presence_of_element_located((By.XPATH, '//input[@name="password"]'))
        )
    username_input.send_keys(username)
    password_input.send_keys(password)

    sleep(random.uniform(2,3)) 

    button = WebDriverWait(driver, 10).until(   
            EC.presence_of_element_located((By.XPATH, '//button[@type="submit"]'))
        )
    button.click()

    sleep(random.uniform(2,3))


def search_instagram(driver, HASHTAG):
    search_input =  WebDriverWait(driver, 10).until(   
            EC.presence_of_element_located((By.XPATH, '//input[@placeholder="Busca"]'))
        )
    search_input.send_keys(HASHTAG)

    sleep(random.uniform(2,3))

    search_link = WebDriverWait(driver, 10).until(   
            EC.presence_of_element_located((By.XPATH, f'//a[@href="/explore/tags/{HASHTAG.replace("#", "")}/"]'))
        )
    search_link.click()

    sleep(random.uniform(4,5))


def scraping_instagram(driver, debug_print=False):
    """
    driver: selenium driver
    debug_print: allows printing exception
    """
    data = []

    # Scrolling
    for i in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(random.uniform(2,3))
    
    posts = driver.find_elements(By.XPATH, '//article//a')

    print(f"Estimated time to end: {len(posts)*4} seconds")

    for post in posts:
        try:
            driver.execute_script("arguments[0].click();",post) 
        
            user = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, '//article[@role="presentation"]//h2//a'))).text
            description = cleaning_text(driver.find_element(By.XPATH, '//article[@role="presentation"]//h2/following-sibling::span').text)
            href = post.get_attribute('href')

            try:
                hashtags = driver.find_elements(By.XPATH, '//article[@role="presentation"]//h2/following-sibling::span/a[@class != "notranslate"]')
                hashtags = [cleaning_text(h.text) for h in hashtags]
            except Exception as e:
                if debug_print: print(f'***** HASTAG {href} *****', e)
                hashtags = []
            
            try:
                labels = driver.find_elements(By.XPATH, '//article[@role="presentation"]//h2/following-sibling::span/a[@class="notranslate"]')
                labels = [cleaning_text(l.text) for l in labels]
            except Exception as e: 
                if debug_print: print(f'***** LABELS {href} *****', e)
                labels= []

            try:
                likes = driver.find_element(By.XPATH, '//article[@role="presentation"]//a[@class="zV_Nj"]/span').text
            except Exception as e:
                if debug_print: print(f'***** LIKES {href} *****', e)
                likes = 0

            data.append({
                "post_link": href,
                "user": user,
                "likes": likes,
                "labels":labels,
                "hashtag":hashtags,
                "description": description
            })

            # We use driver here, because post is limited to their scope 'a'
            close_button = WebDriverWait(driver, 10).until(   
                EC.presence_of_element_located((By.XPATH, '//div[@class="NOTWr"]/button'))
            )
            sleep(random.uniform(0.3,1.2))
            close_button.click()

        except Exception as e:
            if debug_print: print(f'***** POST {href}  *****', e)
            continue
        
    df = pd.DataFrame(data)
    return df

def save_dataframe_to_csv(df):
    df.to_csv(r'data.csv', header=True, index=True, sep='~')

#### FUNCION TRIGGER QUE LANZA EL PROCESO

In [77]:
def run_scraping_process(HASHTAG = '#peru'):
    driver = get_driver()
    logging_instagram(driver)
    search_instagram(driver, HASHTAG)
    df = scraping_instagram(driver)
    save_dataframe_to_csv(df)
    return df

In [78]:
HASHTAG = '#cuzco'
data = run_scraping_process(HASHTAG)
data

C:\Users\AlonsoPC\AppData\Local\Temp\ipykernel_19312\1738824876.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe', options = options)


Estimated time to end: 192 seconds


,post_link,user,likes,labels,hashtag,description
0,https://www.instagram.com/p/CYaH5HjOMbd/,pahfm,112,[],"[#peru, #cuzco, #cuscodestinos, #cuscoentusojo...",Pensando que horas o mal de altitude vai passa...
1,https://www.instagram.com/p/CZHKjwjgW-U/,supermyha,228,[],[],Цены в Перу Пробежимся по перуанским ценам да...
2,https://www.instagram.com/p/CYNkSC4roBo/,darwinrengifo,173,[],"[#2022, #happynewyear, #newyear, #añonuevo, #c...",1ER DÍA DEL AÑO 2022🗒✅ . Quedan 364 páginas má...
3,https://www.instagram.com/p/CX9XoXnvC7y/,aletellog,175,[],"[#cuscoperu, #sacsayhuaman, #cuzco]",Manifestando viajes que me llenen el alma 🍃 (...
4,https://www.instagram.com/p/CZHPn72sB3g/,bulutxin,121,"[@zebra_jp_official, @lefrancbourgeoisofficiel...","[#urbansketching, #urbansketcher, #urbansketch...","A new page in my small squared ""cathedrals' sk..."
5,https://www.instagram.com/p/CYxWXjJsviJ/,viajeroen360,369,[],"[#cusco, #cuzco, #cuscoperu, #viajes, #turismo...",Cocalmayo: las increíbles pozas en Cusco a las...
6,https://www.instagram.com/p/CZFxoh4qheH/,perutravelspecialist,162,[@viajandomaeefilha],"[#maeefilha, #cuscocity, #cuzco, #cuzcoperu, #...",DIVIRTA-SE! Aproveite cada momento com quem vo...
7,https://www.instagram.com/p/CYVB_aBsA8z/,andres_cantos95,210,[],"[#cuzco, #peru, #sacsayhuaman, #travel]",Sacsayhuamán es la segunda edificación inca ma...
8,https://www.instagram.com/p/CY4RDodrhLo/,sebaseldelmapa,1.335,[],"[#machupicchu, #cuzco, #travelgram]",🗻Mucho se sabe que Machu Picchu es una de las ...
9,https://www.instagram.com/p/CZKTHvhrkMo/,intira_gin,7,[],"[#intiragin, #perugin, #almaandina, #cuscogin,...",𝘐𝘯𝘵𝘪𝘳𝘢 𝘯𝘢𝘤𝘦 𝘢 𝟸.𝟾𝟶𝟶 𝘮.𝘴.𝘯.𝘮. 𝘴𝘪𝘯𝘰́𝘯𝘪𝘮𝘰 𝘥𝘦 𝘊𝘶𝘴𝘲...
